In [637]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dataframe_image as dfi
from datetime import datetime
import scipy
import itertools
import geopandas as gpd
import xarray as xr
import regionmask
import statsmodels.api as sm
import statsmodels.formula.api as smf

Data from POWER Data Access Viewer - NASA (1997-2020)

In [638]:
da1 = r"C:\Users\PcLaptop\Documents\GitHub\Climate-and-conflict\Datasets\POWER_Regional_monthly_1997_2020.nc"
da2 = r"C:\Users\PcLaptop\Documents\GitHub\Climate-and-conflict\Datasets\POWER_Regional_monthly_1997_2020_south.nc"

file_paths_list =[da1,da2]
monthly_forecast=xr.Dataset()

for file in file_paths_list:
        monthly_forecast = xr.merge([monthly_forecast,xr.open_mfdataset(file)], compat='no_conflicts')

In [639]:
# rename 'T2M_MAX':'tmx' in monthly_forecast
monthly_forecast = monthly_forecast.rename({'T2M_MAX':'tmx','PRECTOTCORR':'pre'})

In [640]:
#remove the extra month
monthly_forecast = monthly_forecast.sel(time=~((monthly_forecast.time.astype(str).str[4:6] == '13') ))

Data on conflict events from ACLED

In [641]:
file = r"C:\Users\PcLaptop\Documents\GitHub\Climate-and-conflict\Datasets\ACLED_1997-01-01-2023-07-18_Somalia.csv"
df = pd.read_csv(file)

#remove events types of strategic developments
df = df[df['event_type'] != 'Strategic developments']

Shapefile with administrative boundaries of Somalia

In [642]:
path = r"C:\Users\PcLaptop\Documents\GitHub\Climate-and-conflict\Datasets\som_adm_ocha_itos_20230308_shp\som_admbnda_adm1_ocha_20230308.shp"
states_gdf = gpd.read_file(path) 

Limit the lat-lon and time

In [643]:
def get_aoi(shp, world=True):
    lon_lat = {}
    # Get lat min, max
    aoi_lat = [float(shp.total_bounds[1]), float(shp.total_bounds[3])]
    aoi_lon = [float(shp.total_bounds[0]), float(shp.total_bounds[2])]

    lon_lat["lon"] = aoi_lon
    lon_lat["lat"] = aoi_lat
    return lon_lat

bounds = get_aoi(states_gdf)

In [644]:
start_date = '1980-01-01'
end_date = '2020-12-31'

region = monthly_forecast[["pre",'tmx']].sel(
    time=slice(start_date, end_date),
    lon=slice(bounds["lon"][0], bounds["lon"][1]),
    lat=slice(bounds["lat"][0], bounds["lat"][1]))

In [645]:
region_mask = regionmask.mask_3D_geopandas(states_gdf,
                                         monthly_forecast.lon,
                                         monthly_forecast.lat)

temp_pre = region.where(region_mask)

In [646]:
temp_pre['time']=temp_pre['time'].astype(str)
temp_pre = temp_pre.groupby("time").mean(["lat", "lon"]).to_dataframe().reset_index()
temp_pre['time'] = temp_pre['time'].str[:4] + '-' + temp_pre['time'].str[4:6]

In [647]:
replacement_dict = {0  :  'Awdal',
1    :         'Bakool',
2      :       'Banadir',
3      :         'Bari',
4       :         'Bay',
5        :  'Galgaduud',
6          :      'Gedo',
7          :   'Hiraan',
8   :       'Lower_Juba',
9   :   'Lower_Shabelle',
10  :      'Middle_Juba',
11   : 'Middle_Shabelle',
12    :          'Mudug',
13    :        'Nugaal',
14      :       'Sanaag',
15       :        'Sool',
16        :   'Togdheer',
17   : 'Woqooyi_Galbeed'}

temp_pre['admin1'] = temp_pre['region'].replace(replacement_dict)
df['admin1'] = df['admin1'].str.replace(' ', '_')
temp_pre.drop('region', axis=1, inplace=True)

In [648]:
# Modify event_date column to datetime

df['event_date'] = pd.to_datetime(df['event_date'])
df = df.set_index('event_date') 

In [649]:
conflict = df.groupby([pd.Grouper(freq='M'),"admin1"]).count()
conflict.reset_index(level=[0, 1], inplace=True)
conflict = conflict[['event_date','admin1','year']].rename(columns={'year': 'conflicts','event_date': 'time'})

# Aggregate the datetime objects by month
conf = conflict.groupby([pd.Grouper(key='time', freq='M'),'admin1'])['conflicts'].sum().to_frame()

In [650]:
# Reindex the DataFrame with all dates and districts and fill missing values with 0

dates = conf.index.get_level_values('time').unique()
districts = conf.index.get_level_values('admin1').unique()
all_combinations = pd.MultiIndex.from_product([dates, districts], names=['time', 'admin1'])

conflicts = conf.reindex(all_combinations, fill_value=0).reset_index()    
conflicts = conflicts.sort_values(by=['time', 'admin1'], ascending=[True, True])
conflicts.reset_index(drop=True, inplace=True)

In [651]:
# Add Banadir region with tmx and pre as mean of the neighbouring regions

district1 = 'Lower_Shabelle'  
district2 = 'Middle_Shabelle'  

# Calculate the mean tmx and pre for the neighboring districts
mean_t = temp_pre[(temp_pre['admin1']==district1) | (temp_pre['admin1']==district2)].groupby('time')['tmx'].mean()
mean_p = temp_pre[(temp_pre['admin1']==district1) | (temp_pre['admin1']==district2)].groupby('time')['pre'].mean()

new_data = pd.DataFrame({ 'admin1': 'Banadir', 'tmx': mean_t, 'pre': mean_p}).reset_index()

# Append the new DataFrame to the original DataFrame
df3 = pd.concat([temp_pre, new_data])
temp_pre = df3.sort_values(by=['time', 'admin1'], ascending=[True, True]).reset_index(drop=True)

In [652]:
temp_pre['month'] = temp_pre['time'].str[5:7]
conflicts['time'] = conflicts['time'].dt.strftime('%Y-%m').values
temp_pre = temp_pre[['time','admin1','tmx','pre']]

In [653]:
# Split the dataframe into regions

reg=[]
for admin in temp_pre['admin1'].unique():
    a = temp_pre[temp_pre['admin1']==admin].reset_index(drop=True)
    reg.append(a)

In [654]:
# Calculate the TA (temperature anomaly), PA (precipitation anomaly) and DL (drought lenght) for each region

avg_t = avg_p = std_t = std_p = np.zeros(18)

for i in range(18):

    reg[i]['year'] , reg[i]['month'] = reg[i]['time'].str[:4] , reg[i]['time'].str[5:7]

    # TA
    mean_temp_i , std_temp_i  = reg[i].groupby('month')['tmx'].mean() , reg[i].groupby('month')['tmx'].std()
    reg[i]['avg_temp'] , reg[i]['std_temp']  = reg[i]['month'].map(mean_temp_i) , reg[i]['month'].map(std_temp_i)
    reg[i]['diff_t']= (reg[i]['tmx']-reg[i]['avg_temp'])/reg[i]['std_temp']
    reg[i]['TA'] = (reg[i]['diff_t'].shift(2) + reg[i]['diff_t'].shift(1) + reg[i]['diff_t'])/3

    # PA
    mean_pre_i , std_pre_i  = reg[i].groupby('month')['pre'].mean() , reg[i].groupby('month')['pre'].std()
    reg[i]['avg_pre'] , reg[i]['std_pre']= reg[i]['month'].map(mean_pre_i) , reg[i]['month'].map(std_pre_i)
    reg[i]['diff_p']= (reg[i]['pre']-reg[i]['avg_pre'])/reg[i]['std_pre']
    reg[i]['PA'] = (reg[i]['diff_p'].shift(2) + reg[i]['diff_p'].shift(1) + reg[i]['diff_p'])/3
    
    # DL 
    reg[i]['DL'] = 0
    mask = reg[i]['TA'] > 0
    group_id = (mask != mask.shift()).cumsum()             # Create a group identifier for each consecutive group
    count = reg[i].groupby(group_id).cumcount() + 1        # Calculate the count within each group
    reg[i]['DL'] = np.where(mask, count, 0)                # Assign the count values to the 'DL' column

    # add columns TA_lag1, TA_lag2, TA_lag3, TA_lag4, PA_lag1, PA_lag2, DL_lag1, DL_lag2, DL_lag3, DL_lag4, DL_lag5
    # reg[i]['TA_lag1'] = reg[i]['TA'].shift(1)
    # reg[i]['TA_lag2'] = reg[i]['TA'].shift(2)
    # reg[i]['TA_lag3'] = reg[i]['TA'].shift(3)
    # reg[i]['TA_lag4'] = reg[i]['TA'].shift(4)
    # reg[i]['TA_lag5'] = reg[i]['TA'].shift(5)

    # reg[i]['PA_lag1'] = reg[i]['PA'].shift(1)
    # reg[i]['PA_lag2'] = reg[i]['PA'].shift(2)
    # reg[i]['PA_lag3'] = reg[i]['PA'].shift(3)
    # reg[i]['PA_lag4'] = reg[i]['PA'].shift(4)
    # reg[i]['PA_lag5'] = reg[i]['PA'].shift(5)

    # reg[i]['DL_lag1'] = reg[i]['DL'].shift(1)
    # reg[i]['DL_lag2'] = reg[i]['DL'].shift(2)
    # reg[i]['DL_lag3'] = reg[i]['DL'].shift(3)
    # reg[i]['DL_lag4'] = reg[i]['DL'].shift(4)
    # reg[i]['DL_lag5'] = reg[i]['DL'].shift(5)

    reg[i] = reg[i].reset_index()

In [655]:
temp_pre_c = pd.concat([reg[i] for i in range(18)], axis=0)
temp_pre_c = temp_pre_c.dropna()
# Select a subset of the dataframes from 1997-01 to 2009-12

start='1997-01'
end='2009-12'
temp_pre_97_09 = temp_pre_c[(temp_pre_c['time'] >= start) & (temp_pre_c['time'] <= end)]
conflicts_97_09 = conflicts[(conflicts['time'] >= start) & (conflicts['time'] <= end)]

df_c_97_09 = pd.merge(temp_pre_97_09, conflicts_97_09, on=['time','admin1'], how='outer')
df_c_97_09 = df_c_97_09.fillna(0)
df_c_97_09 = df_c_97_09.drop(['avg_temp', 'avg_pre', 'std_temp', 'std_pre', 'diff_t', 'diff_p', 'tmx', 'pre'], axis=1)
df_c_97_09 = df_c_97_09.sort_values(by=['time','admin1'], ascending=[True, True]).reset_index(drop=True)

In [668]:
# Create the dummy variables

#one for each country
df_dummies = pd.get_dummies(df_c_97_09['admin1'])
df_with_dummies = df_c_97_09.join(df_dummies)

#one for each month
df_c_97_09['month'] = pd.DatetimeIndex(df_c_97_09['time']).month_name()
df_dummies_m = pd.get_dummies(df_c_97_09['month'])
df_with_dummies = df_with_dummies.join(df_dummies_m)
df_with_dummies['month'] = pd.DatetimeIndex(df_c_97_09['time']).month

#one for each for each country-month pair
df_dummies_mr = pd.get_dummies(df_c_97_09['admin1'] + df_c_97_09['month'])
df_with_dummies = df_with_dummies.join(df_dummies_mr)
df_with_dummies = df_with_dummies.replace({True: 1, False: 0})

#create a dictionary where the keys are increasing integers and the values are the values of the time column
time_dict = dict(enumerate(df_c_97_09['time'].unique(), 25))
inv_time_dict = {v: k for k, v in time_dict.items()}

#create a new variable for the month_year column
df_c_97_09['time'].replace(inv_time_dict, inplace=True)

df_dummies_new = pd.get_dummies(df_c_97_09['time'].astype(str), drop_first=True)
df_dummies_new = df_dummies_new.replace({True: 1, False: 0})
df_with_dummies = df_with_dummies.join(df_dummies_new)

In [669]:
y_var_name = 'conflicts'
X_var_names = ['TA','PA','DL']

In [670]:
# Regression expression for OLS with dummies

unit_names = df_c_97_09['admin1'].unique().tolist()
unit_names.sort()
unit_names_t = df_c_97_09['month'].unique().tolist()
unit_names_my = df_c_97_09['time'].astype(str).unique().tolist()
unit_names_mr = (df_c_97_09['admin1'] + df_c_97_09['month']).unique().tolist()
unit_names_myr = (df_c_97_09['admin1'] + df_c_97_09['time'].astype(str)).unique().tolist()

lsdv_expr = y_var_name + ' ~ '
i = 0
for X_var_name in X_var_names:
    if i > 0:
        lsdv_expr = lsdv_expr + ' + ' + X_var_name
    else:
        lsdv_expr = lsdv_expr + X_var_name
    i = i + 1
#for dummy_name in unit_names[:-1]:
  # lsdv_expr = lsdv_expr + ' + ' + dummy_name
#or dummy_name_t in unit_names_t[:-1]:
   #lsdv_expr = lsdv_expr + ' + ' + dummy_name_t
for dummy_name_mr in unit_names_mr[:-1]:
    lsdv_expr = lsdv_expr + ' + ' + dummy_name_mr
#lsdv_expr = lsdv_expr + ' - ' + '1'
print('Regression expression for OLS with dummies=' + lsdv_expr)

Regression expression for OLS with dummies=conflicts ~ TA + PA + DL + AwdalJanuary + BakoolJanuary + BanadirJanuary + BariJanuary + BayJanuary + GalgaduudJanuary + GedoJanuary + HiraanJanuary + Lower_JubaJanuary + Lower_ShabelleJanuary + Middle_JubaJanuary + Middle_ShabelleJanuary + MudugJanuary + NugaalJanuary + SanaagJanuary + SoolJanuary + TogdheerJanuary


In [666]:
lsdv_model = smf.ols(formula=lsdv_expr, data=df_with_dummies)
lsdv_model_results = lsdv_model.fit()
print(lsdv_model_results.summary())

                            OLS Regression Results                            
Dep. Variable:              conflicts   R-squared:                       0.295
Model:                            OLS   Adj. R-squared:                  0.287
Method:                 Least Squares   F-statistic:                     37.17
Date:                Thu, 14 Sep 2023   Prob (F-statistic):          1.89e-183
Time:                        15:56:45   Log-Likelihood:                -8751.2
No. Observations:                2790   AIC:                         1.757e+04
Df Residuals:                    2758   BIC:                         1.776e+04
Df Model:                          31                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           0.1497      0.588     

In [667]:
#compute AIC
print('AIC = ' + str(lsdv_model_results.aic))

AIC = 17566.42583986604


In [661]:
dist = pd.read_csv(r"C:\Users\PcLaptop\Documents\GitHub\Climate-and-conflict\dist_som.csv")

In [662]:
inv_dist = 1/(dist+0.001)
inv_dist.reset_index(inplace=True)
inv_dist['index'] = inv_dist['index'].replace(replacement_dict)

In [663]:
#df_new=df_with_dummies.merge(inv_dist, left_on='admin1', right_on='index')

In [664]:
df_with_dummies.drop(['index'], axis=1, inplace=True)